In [ ]:
import requests as rs
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandas import DataFrame as dtf
import re
from datetime import datetime as dt
from IPython.display import display as dp

# Trade

In [ ]:
class Trade:

  def __init__(self, crypto = '', save = False) -> None:
    self.crypto = crypto.upper() if crypto else 'DOGE'
    self.save = save

  def get_current_prices(self):

    url = 'https://www.binance.com/en/markets?p'

    response = rs.get(url)
    if response.status_code != 200:            return 'Not [200]'

    soup = bs(response.content, 'html.parser')

    self.df = dtf({
        'full_name':      [name.string for name in soup.find_all(class_ = 'css-1ap5wc6')],
        'abb':            [abb.string for abb in soup.find_all(class_ = 'css-1x8dg53')],
        'price':          [price.string for price in soup.find_all(class_ = 'css-ovtrou')]
        })

    try:                  self.df['price_float'] = [float(re.sub('[\$\,]', '', price)) for price in self.df.price]
    except:               return "Can't convert the price into float!"

    try:                  ex_crypto = float(self.df[self.df['abb'] == self.crypto]['price_float'])
    except:               print("The wanted asset [%s] haven't been found!" % self.crypto);   return self.df
                         
    self.df[ self.crypto + '\n' + 
            dt.today().strftime('%d %b %Y') ]   =  [
                 ex_crypto / price for price in self.df['price_float']]



    if self.save:

      try:                self.df.to_csv('/content/drive/MyDrive/trade/%s%s.csv' % (self.crypto.lower(), dt.today().strftime('_%d%m%y')))
      except:             print('An error occured => [save]!')

    return self.df

# Profit

In [ ]:
class Profit: 

  def __init__(self, crypto, history, return_ = False, select = False, only_prof = False):
    self.history = history
    self.crypto = crypto.upper()
    self.return_ = return_
    self.select = select
    self.only_prof = only_prof

  def show_profit(self):

    path = self.crypto.lower() + self.history.strftime('_%d%m%y')
    tr_ = (self.crypto, self.history.strftime('%d %b %Y'))
    cr_ = (self.crypto, dt.today().strftime('%d %b %Y'))
    cur = Trade(self.crypto.lower()).get_current_prices()

    if not len(cur):        return 
    print(('/content/drive/MyDrive/trade/%s.csv' % path))
    try:                    trade = pd.read_csv('/content/drive/MyDrive/trade/%s.csv' % path); print('\ttrade retrieved!')
    except:                 print("The file can't be opened. Check if the directory or file name is correct!"); return

    try:                    merged = pd.merge(trade[['full_name', 'abb', 'price', 'price_float', '%s\n%s' % tr_]
                                                    ],cur[['price', 'abb', 'price_float', '%s\n%s' % cr_]
                                                          ], on='abb', suffixes = ['_trade', '_cur']); print('\tfiles merghed!')
    except:                 print("Merging failed!");                                     return
    
    print('%s\n%s\n' % cr_ + '%s\n%s' % tr_)

    try:                    merged['cur_prof'] = merged[
        '%s\n%s_cur' % cr_ if cr_ == tr_ else '%s\n%s' % cr_] - merged[
            '%s\n%s_trade' % tr_ if cr_ == tr_ else '%s\n%s' % tr_]; print('\tCurrent profit found!') 

    except:                 print('Current profit observation failed!');                  return

    try:                    merged['cur_prof_percent'] = merged[
        '%s\n%s_cur' % cr_ if cr_ == tr_ else '%s\n%s' % cr_] / merged[
            '%s\n%s_trade' % tr_ if cr_ == tr_ else '%s\n%s' % tr_] * 100 - 100; print('\tCurrent profit persentage found!')
    except:                 print('Current profit percentage observation failed!');       return

    try:                    merged.drop(['price_float_trade','price_float_cur'], axis=1, inplace=True); print("['price_float_trade','price_float_cur'] dropped!")
    except:                 print("Couldn't drop columuns!");                             return


    if not self.select and not self.only_prof and self.return_:                           return merged[
        
        ['full_name', 'abb', 
         'price_trade', 
         'price_cur', 
         '%s\n%s_trade' % tr_ if cr_ == tr_ else '%s\n%s' % tr_, 
         '%s\n%s_cur' % cr_ if cr_ == tr_ else '%s\n%s' % cr_, 
         'cur_prof', 
         'cur_prof_percent']

    ]

    if not self.select and self.only_prof:                                                return merged[
        
        merged.cur_prof > 0

    ]

    if self.select:         target = input('Asset: ').upper()

    try:                    selection = merged[merged['abb'] == target];                  return selection
    except:                 print("Asset not found!");                                    return

    


# Run


In [ ]:
# Trade

df = Trade(input('Asset: '), save = bool(input('Save? 1/0: '))).get_current_prices()

Asset: busd
Save? 1/0: 1


In [ ]:
# Profit

def profit():
  try:
    return Profit(input('Asset: '),
                  dt(int(input('year: ')), int(input('month: ')), int(input('day: '))),
                  return_=True, select=False, only_prof=False).show_profit()
  except: 
    print('Error occured!')
2
dp(profit())

In [ ]:
# Profit

def profit():
  try:
    return Profit('doge',
                  dt(2022, 12, 30),
                  return_=1, select=1, only_prof=False).show_profit()
  except: 
    print('Error occured!')

dp(profit()) 

/content/drive/MyDrive/trade/doge_301222.csv
	trade retrieved!
	files merghed!
DOGE
01 Jan 2023
DOGE
30 Dec 2022
	Current profit found!
	Current profit persentage found!
['price_float_trade','price_float_cur'] dropped!
Asset: twt


,full_name,abb,price_trade,DOGE\n30 Dec 2022,price_cur,DOGE\n01 Jan 2023,cur_prof,cur_prof_percent
44,Trust Wallet Token,TWT,$1.31,0.05326,$1.41,0.04995,-0.003309,-6.213323
